### Load libraries and files

In [1]:
import pickle
import pandas as pd
import json
import langdetect
import re
import Algorithmia
import bs4 as bs
import lxml
from urllib.request import Request, urlopen
import time
from fuzzywuzzy import process
import tfidf_matcher as tm
from fake_useragent import UserAgent
import numpy as np
import pandas as pd
import recordlinkage

C:\Users\ACER\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
articles = pd.read_csv('../Data/articles_v1.csv')

In [3]:
articles

,author,date,description,full_text,site_name,title,topic,url,author_article_count,author_name_clean
0,Aja Styles,2020-12-23,Perth mother Clare has found herself mostly co...,Perth mother Clare* has found herself mostly ...,Brisbane Times,'Pack Lego': Perth family caught in hard borde...,entertainment,https://www.brisbanetimes.com.au/national/west...,14,Aja Styles
1,Jake Johnson,2020-12-23,The billâs gifts to the wealthy underscore t...,In late-night votes just hours after nearly 5...,Truthout,Congress Passes COVID Relief With Billions in ...,politics,https://truthout.org/articles/congress-passes-...,33,Jake Johnson
2,Christine Favocci,2020-12-23,It is naive to think that either party is free...,The left has insisted that voter fraud is jus...,The Western Journal,PA Man Facing Charges of Unlawful Voting After...,tech,https://www.westernjournal.com/pa-man-facing-c...,19,Christine Favocci
3,William Rivers Pitt,2020-12-23,What Trump may do in his waning days is only u...,"The endgame being played out by Donald Trump,...",Truthout,What Will Trump Attempt in His Last Days? We M...,politics,https://truthout.org/articles/what-will-trump-...,14,William Rivers Pitt
4,Amy Goodman,2020-12-23,Critics say the $900 billion relief package do...,As Congress passes a $900 billion coronavirus...,Truthout,The Insufficient COVID Stimulus Must Not Be Fo...,business,https://truthout.org/video/the-insufficient-co...,19,Amy Goodman
...,...,...,...,...,...,...,...,...,...,...
119946,Daniel Smith,2020-12-05,Ashley Towne decided to get in shape to marry ...,When you subscribe we will use the informatio...,WalesOnline,'I can't get married looking like this' Rower ...,sport,https://www.walesonline.co.uk/news/uk-news/i-c...,147,Daniel Smith
119947,Victoria Jones,2020-12-05,Experimenting on the ISS allows scientists to ...,When you subscribe we will use the informatio...,WalesOnline,Space experiment could unlock resources for mi...,tech,https://www.walesonline.co.uk/news/uk-news/spa...,92,Victoria Jones
119948,Nisha Mal,2020-12-05,All of the properties are Grade I listed build...,Three buildings featured in a Jane Austen nov...,WalesOnline,Three buildings featured in a Jane Austen nove...,tech,https://www.walesonline.co.uk/news/uk-news/thr...,66,Nisha Mal
119949,Nisha Mal,2020-12-05,"'It's all one big conundrum,' says Sheila Herbert",Woman's home is in Tier 2 while her garden fa...,WalesOnline,Woman's home is in Tier 2 while her garden fal...,tech,https://www.walesonline.co.uk/news/uk-news/wom...,66,Nisha Mal


### Collecting outlet Muckrack URLs

In [4]:
outlets = pd.DataFrame(articles.groupby('site_name')['author_name_clean'].nunique()).reset_index().rename(columns = {'author_name_clean': 'unique_journalist_count'})

In [5]:
outlets

,site_name,unique_journalist_count
0,AZFamily,15
1,Allure,6
2,Architectural Digest,2
3,Arizona Daily Star,13
4,Arkansas Online,43
...,...,...
131,nydailynews.com,68
132,orlandosentinel.com,124
133,seattlepi.com,95
134,sun-sentinel.com,98


In [6]:
len(articles[articles.site_name == 'seattlepi.com'].author_name_clean.unique())

95

In [7]:
len(articles[articles.site_name == 'AZFamily'].author_name_clean.unique())

15

In [8]:
outlets['muckrack_url'] = np.nan

In [9]:
outlets

,site_name,unique_journalist_count,muckrack_url
0,AZFamily,15,NaN
1,Allure,6,NaN
2,Architectural Digest,2,NaN
3,Arizona Daily Star,13,NaN
4,Arkansas Online,43,NaN
...,...,...,...
131,nydailynews.com,68,NaN
132,orlandosentinel.com,124,NaN
133,seattlepi.com,95,NaN
134,sun-sentinel.com,98,NaN


In [10]:
client = Algorithmia.client('simKUmpUiy7PIeS1Sh7EV/cBoMJ1')
algo = client.algo('specrom/Google_scraper/0.1.4')
algo.set_options(timeout=300) # optional

In [11]:
# for i in range(len(outlets)):
#     query = {"query": f'"media-outlet" site: muckrack {outlets.iloc[i].site_name}'}
#     # print(query)
#     try:
#         res = algo.pipe(query).result[0]['url']
#         outlets.loc[i, 'muckrack_url'] = res
#     except:
#         pass
# #     else:
# #         print('none')

### Collecting outlet website URLs

In [14]:
outlets['website_url'] = np.nan

In [15]:
outlets

,site_name,unique_journalist_count,muckrack_url,website_url
0,AZFamily,15,https://muckrack.com/media-outlet/azfamily,NaN
1,Allure,6,https://muckrack.com/media-outlet/allurekorea,NaN
2,Architectural Digest,2,https://muckrack.com/media-outlet/architectura...,NaN
3,Arizona Daily Star,13,https://muckrack.com/media-outlet/tucson,NaN
4,Arkansas Online,43,https://muckrack.com/media-outlet/arkansasdg,NaN
...,...,...,...,...
131,nydailynews.com,68,https://muckrack.com/media-outlet/nydn,NaN
132,orlandosentinel.com,124,https://muckrack.com/media-outlet/orlandosentinel,NaN
133,seattlepi.com,95,https://muckrack.com/media-outlet/seattlepi,NaN
134,sun-sentinel.com,98,https://muckrack.com/media-outlet/sunsentinel,NaN


In [19]:
ua = UserAgent()
for i in range(len(outlets)):
    print(i)
    url = outlets.iloc[i].muckrack_url
    try:
        req = Request(url, headers = {'User-Agent': ua.random})
        sauce = urlopen(req).read()
        soup = bs.BeautifulSoup(sauce,'lxml')
        link = soup.find_all(class_ = 'mr-contact-item-inner')[0].text
        outlets.loc[i, 'website_url'] = link
    except:
        pass

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135


In [22]:
outlets.website_url.value_counts()

http://thenextweb.com/                 5
http://qz.com/                         4
http://chron.com/                      2
http://livemint.com/                   2
http://hindustantimes.com/             2
                                      ..
http://advocate.com/                   1
http://westernjournal.com              1
http://theage.com.au                   1
http://host.madison.com                1
https://us.cnn.com/cnn-underscored/    1
Name: website_url, Length: 124, dtype: int64

### Collecting outlet Alexa rankings and popularity

In [27]:
outlets['popularity'] = 0
outlets['reach_rank'] = 0
outlets['rank_delta'] = 0
outlets['country_rank'] = 0

In [28]:
outlets

,site_name,unique_journalist_count,muckrack_url,website_url,popularity,reach_rank,rank_delta,country_rank
0,AZFamily,15,https://muckrack.com/media-outlet/azfamily,http://azfamily.com,0,0,0,0
1,Allure,6,https://muckrack.com/media-outlet/allurekorea,http://allurekorea.com,0,0,0,0
2,Architectural Digest,2,https://muckrack.com/media-outlet/architectura...,http://architecturaldigest.com,0,0,0,0
3,Arizona Daily Star,13,https://muckrack.com/media-outlet/tucson,http://tucson.com,0,0,0,0
4,Arkansas Online,43,https://muckrack.com/media-outlet/arkansasdg,http://arkansasonline.com/,0,0,0,0
...,...,...,...,...,...,...,...,...
131,nydailynews.com,68,https://muckrack.com/media-outlet/nydn,http://nydailynews.com/,0,0,0,0
132,orlandosentinel.com,124,https://muckrack.com/media-outlet/orlandosentinel,http://orlandosentinel.com/,0,0,0,0
133,seattlepi.com,95,https://muckrack.com/media-outlet/seattlepi,http://seattlepi.com,0,0,0,0
134,sun-sentinel.com,98,https://muckrack.com/media-outlet/sunsentinel,http://sun-sentinel.com/,0,0,0,0


In [29]:
for i in range(len(outlets)):
    
    url = outlets.iloc[i].website_url
    
    try:
        req = Request('http://data.alexa.com/data?cli=10&dat=s&url={}'.format(url), headers = {'User-Agent': ua.random})
        sauce = urlopen(req).read()
        soup = bs.BeautifulSoup(sauce,'xml')

        try:
            popularity = int(soup.find('POPULARITY')['TEXT'])
            outlets.loc[i, 'popularity'] = popularity
        except:
            pass

        try: 
            reach_rank = int(soup.find('REACH')['RANK'])
            outlets.loc[i, 'reach_rank'] = reach_rank
        except:
            pass

        try: 
            rank_delta = int(soup.find('RANK')['DELTA'])
            outlets.loc[i, 'rank_delta'] = rank_delta
        except:
            pass

        try:
            country_rank = int(soup.find('COUNTRY')['RANK'])
            outlets.loc[i, 'country_rank'] = country_rank
        except:
            pass

    except:
        pass

# print(popularity, reach_rank, rank_delta, country_rank)

In [31]:
outlets.describe()

,unique_journalist_count,popularity,reach_rank,rank_delta,country_rank
count,136.000000,1.360000e+02,1.360000e+02,1.360000e+02,136.000000
mean,44.720588,5.809335e+04,5.180240e+04,1.010555e+04,5663.654412
std,59.815109,4.272732e+05,3.887392e+05,1.341302e+05,12517.635695
min,1.000000,0.000000e+00,0.000000e+00,-1.858960e+05,0.000000
25%,6.000000,1.806500e+03,1.444750e+03,-3.660000e+02,192.500000
50%,17.500000,8.427000e+03,6.550000e+03,2.400000e+01,1958.000000
75%,69.750000,2.841350e+04,2.526975e+04,1.798750e+03,6156.250000
max,344.000000,4.989338e+06,4.539359e+06,1.549384e+06,94272.000000


In [32]:
outlets.to_csv('../Data/outlets_v1.csv', index = False)